In [16]:
import tensorflow as tf
import keras as keras
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product
from sklearn.metrics import accuracy_score

from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os
tf.debugging.set_log_device_placement(True)

In [17]:
# load dataset with pandas
dataframe1 = pd.read_csv('data/input_v2/Presentation_activity.csv', sep=',', header=None)
dataframe2 = pd.read_csv('data/input_v2/Chatting_activity.csv', sep=',', header=None)
dataframe3 = pd.read_csv('data/input_v2/Discussion_activity.csv', sep=',', header=None)
dataframe4 = pd.read_csv('data/input_v2/GroupStudy_activity.csv', sep=',', header=None)

# Show how many episodes 
#print(len(dataframe1))
#print(len(dataframe2))
#print(len(dataframe3))
#print(len(dataframe4))

#Combine all dataframe
dataframe = dataframe1.append(dataframe2,ignore_index=True)
dataframe = dataframe.append(dataframe3,ignore_index=True)
dataframe = dataframe.append(dataframe4,ignore_index=True)

#Change NaN data to 0
dataframe = dataframe.fillna(0)
dataset = dataframe.values

#288, 6
#Seperate input and do one-hot encoding
#X=to_categorical(dataset)
X=dataset
print("The shape of X:", X)

# Construct Y label
arr = [] #  empty regular list
for i in range(len(dataframe1)):
    arr.append(0*np.ones((1)))
for i in range(len(dataframe2)):
    arr.append(1*np.ones((1)))
for i in range(len(dataframe3)):
    arr.append(2*np.ones((1)))
for i in range(len(dataframe4)):
    arr.append(3*np.ones((1)))
np_array = np.array(arr)  # transformed to a numpy array

Y=to_categorical(np_array)
Y=argmax(Y, axis=1)

print("The shape of Y:", Y)

The shape of X: [[6. 7. 4. ... 0. 0. 0.]
 [7. 4. 4. ... 0. 0. 0.]
 [7. 4. 4. ... 0. 0. 0.]
 ...
 [2. 6. 4. ... 0. 0. 0.]
 [2. 4. 4. ... 0. 0. 0.]
 [2. 4. 4. ... 0. 0. 0.]]
The shape of Y: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3]


In [18]:
max_len=724
n_members=7
epochs = 50
batch_size = 64
length=724
n_members=2
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

def model1_made():   
    sequence_input = Input(shape=(max_len,), dtype='int32')
    embedded_sequences = Embedding(8, 128, input_length=max_len, mask_zero = True)(sequence_input)

    lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)
    lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
      (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

    print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

    state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
    state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, state_h)

    dense1 = Dense(20, activation="relu")(context_vector)
    dropout = Dropout(0.2)(dense1)
    output = Dense(4, activation="softmax")(dropout)
    model = Model(inputs=sequence_input, outputs=output)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model
def model2_made():
    sequence_input = Input(shape=(max_len,), dtype='int32')
    embedded_sequences = Embedding(8, 128, input_length=max_len, mask_zero = True)(sequence_input)

    lstm = LSTM(64, dropout=0.5, return_sequences = True)(embedded_sequences)
    lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
      (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

    print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

    state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
    state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, state_h)

    dense1 = Dense(20, activation="relu")(context_vector)
    dropout = Dropout(0.2)(dense1)
    output = Dense(4, activation="softmax")(dropout)
    model = Model(inputs=sequence_input, outputs=output)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model
# normalize a vector to have unit norm
def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result

def ensemweighted(accr,yhats,Y_test):
    r2=Y_test
    # define weights to consider
    w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    best_score, best_weights = 0.0, None
    for weights in product(w, repeat=n_members):
        # skip if all weights are equal
        if len(set(weights)) == 1:
            continue
    # hack, normalize weight vector
        weights = normalize(weights)
    # evaluate weights
        yhats =  array(yhats)
        summed =  tensordot(yhats, weights, axes=((0),(0)))
        result = argmax(summed, axis=1)
        score =  accuracy_score(r2, result)
        if score > best_score:
            best_score, best_weights = score, weights
    
    print('>%s %.3f' % (best_weights, best_score))

In [ ]:
for i in range(1):
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
    model1 = model1_made()
    model1.fit(X_train, Y_train, epochs = epochs, batch_size = batch_size, validation_data=(X_test, Y_test), verbose=1)
    accr1 = model1.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    yhat1 = model1.predict(X_test, verbose=0)
    print("\n BiLSTM 테스트 정확도: %.4f" % (accr1[1]))
    
    model2 = model2_made()
    model2.fit(X_train, Y_train, epochs = epochs, batch_size = batch_size, validation_data=(X_test, Y_test), verbose=1)
    accr2 = model2.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    yhat2 = model2.predict(X_test, verbose=0)
    print("\n LSTM 테스트 정확도: %.4f" % (accr2[1]))
    
    yhats=[yhat1, yhat2]
    accr=[accr1,accr2]
    ensemweighted(accr,yhats,Y_test)
    #ensemweighted(accr,yhats,Y_test)
    #history = model.fit(X_train, Y_train, epochs = 50, batch_size = 256, validation_data=(X_test, Y_test), verbose=1)
    #accr = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    #print("\n 테스트 정확도: %.4f" % accur[1]))

In [ ]:
length=158
allen_length=8
# load dataset
dataframe1 = pd.read_csv('data/input_v2/Task1_activity.csv', sep=',', header=None)
dataframe2 = pd.read_csv('data/input_v2/Task2_activity.csv', sep=',', header=None)
dataframe3 = pd.read_csv('data/input_v2/Task3_activity.csv', sep=',', header=None)
dataframe4 = pd.read_csv('data/input_v2/Task5_activity.csv', sep=',', header=None)

#Combine all dataframe
dataframe = dataframe1.append(dataframe2,ignore_index=True)
dataframe = dataframe.append(dataframe3,ignore_index=True)
dataframe = dataframe.append(dataframe4,ignore_index=True)

#Change NaN data to 0
dataframe = dataframe.fillna(0)
dataset = dataframe.values
X=dataset
print("The shape of X:", X)

# Construct Y label
arr = [] #  empty regular list
for i in range(len(dataframe1)):
    arr.append(0*np.ones((1)))
for i in range(len(dataframe2)):
    arr.append(1*np.ones((1)))
for i in range(len(dataframe3)):
    arr.append(2*np.ones((1)))
for i in range(len(dataframe4)):
    arr.append(3*np.ones((1)))
np_array = np.array(arr)  # transformed to a numpy array

Y=to_categorical(np_array)
Y=argmax(Y, axis=1)

print("The shape of Y:", Y)

#X_train, X_test, Y_train, Y_test, X_duration_train, X_duration_test,Y_duration_train,Y_duration_test = train_test_split(X,Y, X_duration,Y_duration,test_size=0.2)
#print(Y_train.shape) 
#print(Y_duration_train.shape) 


In [ ]:
max_len=158
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(100, 128, input_length=max_len, mask_zero = True)(sequence_input)

lstm = LSTM(64, dropout=0.5, return_sequences = True)(embedded_sequences)
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.2)(dense1)
output = Dense(4, activation="softmax")(dropout)
model = Model(inputs=sequence_input, outputs=output)

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
    history = model.fit(X_train, Y_train, epochs = 1, batch_size = 256, validation_data=(X_test, Y_test), verbose=1)

    print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Prefet


 테스트 정확도: 1.0000
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
E


 테스트 정확도: 1.0000
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
E


 테스트 정확도: 1.0000
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
E

In [16]:
yhat = model.predict(X_test, verbose=0)

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [17]:
print(argmax(yhat, axis=1), Y_test)

[1 2 0 0 3 0 3 1 0 1 1 3 2 3 2 2 1 3 2 2 3] [1 2 0 0 3 0 3 1 0 1 1 3 2 3 2 2 1 3 2 2 3]
